In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_gps_info(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    gps_info = {}
    for tag, value in exif_data.items():
        decoded = TAGS.get(tag, tag)
        if decoded == 'GPSInfo':
            for t in value:
                sub_decoded = GPSTAGS.get(t, t)
                gps_info[sub_decoded] = value[t]
    return gps_info

# Example usage
gps_info = get_gps_info("path_to_your_image.jpg")
print(gps_info)


In [4]:
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from osgeo import gdal, osr
from osgeo import gdal

# Enable exceptions
#gdal.UseExceptions()

# Alternatively, to disable exceptions
gdal.DontUseExceptions()

def get_gps_info(image_path):
    """Extract GPS information from an image file."""
    image = Image.open(image_path)
    exif_data = image._getexif()
    gps_info = {}
    if exif_data:
        for tag, value in exif_data.items():
            decoded = TAGS.get(tag, tag)
            if decoded == 'GPSInfo':
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_info[sub_decoded] = value[t]
    return gps_info

def convert_to_degrees(value):
    """Convert GPS coordinates to degrees."""
    d = float(value[0])
    m = float(value[1])
    s = float(value[2])
    return d + (m / 60.0) + (s / 3600.0)

def georeference_image(input_image, output_image, gps_info):
    """Georeference an image using GPS information and save as GeoTIFF."""
    # Extract latitude and longitude from gps_info
    lat_ref = gps_info.get('GPSLatitudeRef', 'N')
    lon_ref = gps_info.get('GPSLongitudeRef', 'E')
    lat = gps_info.get('GPSLatitude')
    lon = gps_info.get('GPSLongitude')
    
    if not lat or not lon:
        print(f"No GPS data found for {input_image}. Skipping...")
        return

    lat_deg = convert_to_degrees(lat)
    lon_deg = convert_to_degrees(lon)
    
    if lat_ref != 'N':
        lat_deg = -lat_deg
    if lon_ref != 'E':
        lon_deg = -lon_deg

    # Open the image
    ds = gdal.Open(input_image)
    if ds is None:
        print(f"Failed to open {input_image}.")
        return
    
    # Define the GeoTransform (assuming north-up, no rotation)
    gt = [lon_deg, 0.0, 0.0, lat_deg, 0.0, 0.0]
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # WGS84
    
    # Set the geotransform and projection
    ds.SetGeoTransform(gt)
    ds.SetProjection(srs.ExportToWkt())
    
    # Save the GeoTIFF
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.CreateCopy(output_image, ds)
    out_ds = None  # Close the file

def process_directory(input_dir, output_dir):
    """Process all images in a directory and save georeferenced GeoTIFFs."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith(('jpg', 'jpeg')):
            input_image_path = os.path.join(input_dir, file_name)
            output_image_path = os.path.join(output_dir, os.path.splitext(file_name)[0] + '.Gtiff')
            
            gps_info = get_gps_info(input_image_path)
            if gps_info:
                georeference_image(input_image_path, output_image_path, gps_info)
            else:
                print(f"No GPS data for {file_name}. Skipping...")



# Example usage
input_directory = "101MEDIA"
output_directory = "output_geotiff"
process_directory(input_directory, output_directory)
